In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.cluster import KMeans

from xgboost.sklearn import XGBClassifier
from xgboost.core import DMatrix
from xgboost.training import train, cv


C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [470]:
data_path = "dataset/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
df_train = pd.read_json(train_file)
df_test = pd.read_json(test_file)
#df_train = df_test
print(df_train.shape)
print(df_test.shape)

#df_train.index = df_train['listing_id'].astype(int)
#df_test.index = df_test['listing_id'].astype(int)

target_num_map = {'high':0, 'medium':1, 'low':2}
y_train = np.array(df_train['interest_level'].apply(lambda x: target_num_map[x]))
df_train = df_train.drop(['interest_level'], axis=1)


(49352, 15)
(74659, 14)


In [471]:
print('start...')
#df_train.index = df_train['listing_id']
#df_train = df_train.drop(['listing_id'], axis=1)

#df_test.index = df_test['listing_id']
#df_test = df_test.drop(['listing_id'], axis=1)

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

#date
df_all['created'] = pd.to_datetime(df_all['created'])
#year - all 2016
#df_all['created_year'] = df_all['created'].dt.year
df_all['created_month'] = df_all['created'].dt.month
df_all['created_day'] = df_all['created'].dt.day
df_all['created_day_of_year'] = df_all['created'].dt.strftime('%j').astype(int)
df_all['created_hour'] = df_all['created'].dt.hour
df_all['created_weekday'] = df_all['created'].dt.weekday
df_all = df_all.drop(['created'], axis=1)

'''
#create_weekday categorical
ohe = OneHotEncoder(sparse=False)
df_all_ohe = ohe.fit_transform(df_all.created_weekday.reshape(-1, 1)) 	
for i in range(df_all_ohe.shape[1]):
    df_all['ohe' + str(i)] = df_all_ohe[:, i]
df_all = df_all.drop(['created_weekday'], axis=1)
'''

#adjust incorrect x/y
x_mean = df_all.latitude.mean()
y_mean = df_all.longitude.mean()

df_all.loc[df_all.latitude < x_mean - 5, 'latitude'] = x_mean - 5
df_all.loc[df_all.latitude > x_mean + 5, 'latitude'] = x_mean + 5
df_all.loc[df_all.longitude < y_mean - 5, 'longitude'] = y_mean - 5
df_all.loc[df_all.longitude > y_mean + 5, 'longitude'] = y_mean + 5

#log x/y
df_all['logx'] = np.log(df_all['latitude'])
df_all['logy'] = np.log(df_all['longitude'] + 100)

#radius
df_all['radius'] = np.log((df_all.latitude - x_mean)*(df_all.latitude - x_mean) + (df_all.longitude - y_mean)*(df_all.longitude - y_mean))

'''
#zones
n_zones = 88
x_min = df_all.logx.mean() - 0.004
x_max = df_all.logx.mean() + 0.003
y_min = df_all.logy.mean() - 0.003
y_max = df_all.logy.mean() + 0.003

df_all2 = df_all[(df_all.logx >= x_min) & (df_all.logx <= x_max) & (df_all.logy >= y_min) & (df_all.logy <= y_max)]
kmeans = KMeans(n_clusters=n_zones, random_state=0).fit(df_all2[['logx', 'logy']])

print('zones', df_all.shape)

for i in range(n_zones):
    df_all['zone' + str(i)] = 0
    df_all.loc[df_all2.logx[kmeans.labels_ == i].index, 'zone' + str(i)] = 1

print('zones', df_all.shape)
'''

#price
#df_all.loc[df_all['price'] > 100000, 'price'] = 100000

#log price
#df_all['logprice'] = np.log(df_all.price)

df_all["num_photos"] = df_all["photos"].apply(len)
df_all["num_features"] = df_all["features"].apply(len)
df_all["num_description_words"] = df_all["description"].apply(lambda x: len(x.split(" ")))

df_all['features'] = df_all['features'].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(df_all[:df_train.shape[0]]['features'])
te_sparse = tfidf.transform(df_all[df_train.shape[0]:]['features'])

#df_all = df_all.drop(['features', 'description', 'photos'], axis=1)

#categorical features
#categorical = ["display_address", "manager_id", "building_id", "street_address"]
#for f in categorical: 
#    le = LabelEncoder() 
#    df_all[f] = le.fit_transform(df_all[f]) 

print(df_all.shape)

#done
X_train = df_all[:df_train.shape[0]].values
X_test = df_all[df_train.shape[0]:].values

#X_train = csr_matrix(X_train)
#X_test = csr_matrix(X_test)

#X_train = csr_matrix(np.hstack([X_train, tr_sparse.todense()]))
#X_test = csr_matrix(np.hstack([X_test, te_sparse.todense()]))

print('Train', X_train.shape)
print('Test', X_test.shape)

start...
(124011, 24)
Train (49352, 24)
Test (74659, 24)


In [472]:
df_all

,bathrooms,bedrooms,building_id,description,display_address,features,latitude,listing_id,longitude,manager_id,...,created_day,created_day_of_year,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words
0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,...,24,176,7,4,3.706584,3.260306,-7.446341,5,0,95
1,1.0,2,c5c8a357cba207596b04d1afd1e4f130,,Columbus Avenue,Doorman Elevator Fitness_Center Cats_Allowed D...,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,...,12,164,12,6,3.708552,3.259376,-5.639716,11,5,9
2,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,"Top Top West Village location, beautiful Pre-w...",W 13 Street,Laundry_In_Building Dishwasher Hardwood_Floors...,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,...,17,108,3,6,3.707181,3.258027,-5.898510,8,4,94
3,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,Building Amenities - Garage - Garden - fitness...,East 49th Street,Hardwood_Floors No_Fee,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,...,18,109,2,0,3.707552,3.259338,-7.421907,3,2,80
4,1.0,4,0,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,Pre-War,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,...,28,119,1,3,3.709273,3.260045,-4.896160,3,1,68
5,2.0,4,38a913e46c94a7f46ddf19b756a9640c,,West 18th Street,,40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,...,19,110,4,1,3.707282,3.257989,-5.851502,5,0,9
6,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,Stunning unit with a great location and lots o...,West 107th Street,prewar elevator Dogs_Allowed Cats_Allowed LOWR...,40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,...,27,118,3,2,3.708711,3.259403,-5.445840,10,8,87
7,2.0,1,0372927bcb6a0949613ef5bf893bbac7,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,Doorman Elevator Pre-War Terrace Laundry_in_Un...,40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,...,13,104,6,2,3.707277,3.258262,-6.134814,5,8,134
8,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,<p><a website_redacted,Hamilton Terrace,Cats_Allowed Dogs_Allowed Elevator Laundry_In_...,40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,...,20,111,2,2,3.709255,3.260183,-4.910527,5,4,4
9,2.0,4,0,This is a spacious four bedroom with every bed...,522 E 11th,Dishwasher Hardwood_Floors,40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,...,2,93,2,5,3.706911,3.258835,-6.830186,9,2,166


In [6]:
y_train

array([1, 2, 0, ..., 2, 2, 2], dtype=int64)

In [34]:
for x in df_all.columns:
    print(x)

bathrooms
bedrooms
building_id
display_address
latitude
longitude
manager_id
price
street_address
created_month
created_day
created_day_of_year
created_hour
created_weekday
logx
logy
radius
photos_count
features_count
description_words_count


In [297]:
df_all[:10].index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [ ]:
plt.plot(df_all['latitude'], df_all['longitude'], 'ro')
plt.show()

In [268]:
df_all.latitude.describe()

count    124011.000000
mean         40.737641
std           0.744337
min           0.000000
25%          40.728000
50%          40.751700
75%          40.774300
max          44.883500
Name: latitude, dtype: float64

In [305]:
x_mean = df_all.latitude.mean()
df_all[df_all.latitude < x_mean - 5]
#df_all[df_all.latitude > x_mean + 5]


,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,price,street_address,photos_count,features_count,description_words_count,created_month,created_day,created_hour,created_weekday
2787,5.0,6,0,9176,0.0000,0.0000,3423,9995,10787,6,6,303,4,18,2,0
14104,1.0,1,926,13299,0.0000,0.0000,618,1750,3328,8,3,33,5,31,22,1
21715,1.0,1,0,7988,0.0000,0.0000,3959,1725,25472,8,2,40,6,1,19,2
22471,1.0,2,7315,11372,0.0000,0.0000,1167,4395,8007,5,8,63,6,16,15,3
22767,1.0,1,794,9662,34.0126,-118.2710,4148,2425,9226,4,7,26,6,17,2,4
25860,1.0,2,6504,12185,0.0000,0.0000,3215,2950,4768,7,3,24,5,7,8,5
27181,1.0,2,10697,3710,0.0000,0.0000,3818,3619,16820,4,7,161,5,10,10,1
33286,4.0,3,11043,5948,0.0000,0.0000,937,16500,18313,10,13,175,5,11,18,2
34390,1.0,1,3482,16047,0.0000,0.0000,2161,3600,10172,11,7,124,5,10,14,1
35826,1.0,2,3894,1351,0.0000,0.0000,2498,3200,1709,6,4,82,5,24,17,1


In [306]:
df_all.loc[df_all.latitude < x_mean - 5, 'latitude'] = x_mean - 5
df_all.loc[df_all.latitude > x_mean + 5, 'latitude'] = x_mean + 5

In [286]:
df_all.latitude.describe()

count    124011.000000
mean         40.749564
std           0.109465
min          35.737641
25%          40.728000
50%          40.751700
75%          40.774300
max          44.883500
Name: latitude, dtype: float64

In [287]:
df_all.latitude.plot(kind='hist', bins = 100)
plt.show()

In [348]:
np.log(df_all.latitude).describe()

count    124011.000000
mean          1.610121
std           0.001224
min           1.553126
25%           1.609893
50%           1.610146
75%           1.610387
max           1.652087
Name: latitude, dtype: float64

In [350]:
np.log(df_all.latitude).plot(kind='hist', bins = 100)
#np.log10(df_all.latitude).plot(kind='hist', bins = 100)
plt.show()

In [307]:
y_mean = df_all.longitude.mean()
#df_all[df_all.longitude < y_mean - 5]
df_all[df_all.longitude > y_mean + 5]

,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,price,street_address,photos_count,features_count,description_words_count,created_month,created_day,created_hour,created_weekday
2787,5.0,6,0,9176,35.737641,0.0000,3423,9995,10787,6,6,303,4,18,2,0
14104,1.0,1,926,13299,35.737641,0.0000,618,1750,3328,8,3,33,5,31,22,1
21715,1.0,1,0,7988,35.737641,0.0000,3959,1725,25472,8,2,40,6,1,19,2
22471,1.0,2,7315,11372,35.737641,0.0000,1167,4395,8007,5,8,63,6,16,15,3
25860,1.0,2,6504,12185,35.737641,0.0000,3215,2950,4768,7,3,24,5,7,8,5
27181,1.0,2,10697,3710,35.737641,0.0000,3818,3619,16820,4,7,161,5,10,10,1
33286,4.0,3,11043,5948,35.737641,0.0000,937,16500,18313,10,13,175,5,11,18,2
34390,1.0,1,3482,16047,35.737641,0.0000,2161,3600,10172,11,7,124,5,10,14,1
35826,1.0,2,3894,1351,35.737641,0.0000,2498,3200,1709,6,4,82,5,24,17,1
36448,1.0,1,473,12366,35.737641,0.0000,3818,3495,10129,5,2,43,5,10,10,1


In [308]:
df_all.loc[df_all.longitude < y_mean - 5, 'longitude'] = y_mean - 5
df_all.loc[df_all.longitude > y_mean + 5, 'longitude'] = y_mean + 5

In [294]:
df_all.longitude.describe()

count    124011.000000
mean        -73.970473
std           0.116298
min         -78.949434
25%         -73.991700
50%         -73.977800
75%         -73.954700
max         -68.949434
Name: longitude, dtype: float64

In [295]:
df_all.longitude.plot(kind='hist', bins = 100)
plt.show()

In [12]:
np.log(df_all.longitude + 100).describe()

count    124011.000000
mean          3.259222
std           0.004282
min           3.046927
25%           3.258416
50%           3.258950
75%           3.259837
max           3.435617
Name: longitude, dtype: float64

In [13]:
np.log(df_all.longitude + 100).plot(kind='hist', bins = 100)
plt.show()

In [338]:
plt.plot(np.log(df_all['latitude']), np.log(df_all['longitude'] + 100), 'ro')
plt.show()

In [313]:
df_all['logx'] = np.log(df_all['latitude'])
df_all['logy'] = np.log(df_all['longitude'] + 100)

In [321]:
df_all.logx.describe()

count    124011.000000
mean          3.707441
std           0.002819
min           3.576204
25%           3.706916
50%           3.707498
75%           3.708052
max           3.804070
Name: logx, dtype: float64

In [331]:
df_all.logx.plot(kind='hist', bins = 100)
plt.show()

In [13]:
x_min = df_all.logx.mean() - 0.004
x_max = df_all.logx.mean() + 0.003
y_min = df_all.logy.mean() - 0.003
y_max = df_all.logy.mean() + 0.003

In [14]:
print(df_all.shape)
print(df_all[(df_all.logx >= x_min) & (df_all.logx <= x_max) & (df_all.logy >= y_min) & (df_all.logy <= y_max)].shape)

(124011, 19)
(120885, 19)


In [15]:
df_all2 = df_all[(df_all.logx >= x_min) & (df_all.logx <= x_max) & (df_all.logy >= y_min) & (df_all.logy <= y_max)]
plt.plot(df_all2.logx, df_all2.logy, 'ro')
plt.show()

In [563]:
df_all['logx2'] = df_all.apply(lambda row: 0 if (row['logx'] < x_min or row['logx'] > x_max or row['logy'] < y_min or row['logy'] > y_max) else row['logx'], axis=1)
df_all['logy2'] = df_all.apply(lambda row: 0 if (row['logx'] < x_min or row['logx'] > x_max or row['logy'] < y_min or row['logy'] > y_max) else row['logy'], axis=1)

In [578]:
plt.plot(df_all2.logx, df_all2.logy, 'ro')
plt.show()

In [579]:
df_all2[['logx', 'logy']]

,logx,logy
listing_id,,
7211212,3.706584,3.260306
7150865,3.708552,3.259376
6887163,3.707181,3.258027
6888711,3.707552,3.259338
6934781,3.709273,3.260045
6894514,3.707282,3.257989
6930771,3.708711,3.259403
6867392,3.707277,3.258262
6898799,3.709255,3.260183


In [309]:
df_all['radius'] = np.log((df_all.latitude - x_mean)*(df_all.latitude - x_mean) + (df_all.longitude - y_mean)*(df_all.longitude - y_mean))
df_all.radius.describe()

count    124011.000000
mean         -6.132634
std           0.888589
min         -11.586732
25%          -6.713848
50%          -6.109974
75%          -5.631572
max           3.912023
Name: radius, dtype: float64

In [566]:
df_all['radius'].plot(kind='hist', bins = 100)
plt.show()

In [16]:
df_all2.shape

(120885, 19)

In [18]:
n_zones = 20
kmeans = KMeans(n_clusters=n_zones, random_state=0).fit(df_all2[['logx', 'logy']])

In [19]:
len(kmeans.labels_)

120885

In [399]:
sum(kmeans.labels_)

1038103

In [20]:
color = ['bo', 'ro', 'yo', 'go', 'co', 'mo', 'ko', 'bs', 'rs', 'ys', 'gs', 'cs', 'ms', 'ks', 'b^', 'r^', 'y^', 'g^', 'c^', 'm^', 'k^']
for i in range(n_zones):
    plt.plot(df_all2.logx[kmeans.labels_ == i], df_all2.logy[kmeans.labels_ == i], color[i])
plt.show()

In [584]:
for i in range(n_zones):
    print(df_all2.logx[kmeans.labels_ == i].shape)

(12077,)
(12181,)
(1136,)
(10523,)
(2368,)
(9368,)
(3160,)
(4903,)
(9781,)
(8002,)
(1593,)
(1574,)
(4184,)
(1635,)
(992,)
(9512,)
(5596,)
(672,)
(2003,)
(11551,)
(8074,)


In [21]:
df_all2.logx[kmeans.labels_ == 0].index

Int64Index([    12,     15,     20,     21,     22,     38,     66,     69,
               105,    123,
            ...
            123900, 123902, 123906, 123912, 123917, 123918, 123940, 123947,
            123964, 123979],
           dtype='int64', length=11733)

In [22]:
for i in range(n_zones):
    df_all['zone' + str(i)] = 0
    df_all.loc[df_all2.logx[kmeans.labels_ == i].index, 'zone' + str(i)] = 1

In [23]:
sum(df_all['zone0'])

11733

In [555]:
df_all = df_all.drop(['logx2', 'logy2'], axis=1)
df_all.shape

(124011, 39)

In [24]:
df_all

,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,price,street_address,photos_count,...,zone10,zone11,zone12,zone13,zone14,zone15,zone16,zone17,zone18,zone19
0,1.5,3,3797,12282,40.7145,-73.9425,1568,3000,23484,5,...,0,0,1,0,0,0,0,0,0,0
1,1.0,2,8986,9080,40.7947,-73.9667,1988,5465,23680,11,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,8889,13719,40.7388,-74.0018,3733,2850,9827,8,...,0,0,0,1,0,0,0,0,0,0
3,1.0,1,1848,10866,40.7539,-73.9677,282,3275,14237,3,...,0,0,0,0,0,0,0,0,0,0
4,1.0,4,0,15072,40.8241,-73.9493,2618,3350,19227,3,...,0,0,0,0,0,0,1,0,0,0
5,2.0,4,2544,15194,40.7429,-74.0028,3081,7995,15263,5,...,0,0,0,1,0,0,0,0,0,0
6,1.0,2,2711,14964,40.8012,-73.9660,12,3600,7466,10,...,0,0,0,0,0,0,0,0,0,0
7,2.0,1,169,15223,40.7427,-73.9957,3959,5645,4181,5,...,0,0,0,1,0,0,0,0,0,0
8,1.0,1,7635,11646,40.8234,-73.9457,3339,1725,21836,5,...,0,0,0,0,0,0,1,0,0,0
9,2.0,4,0,6459,40.7278,-73.9808,633,5800,20000,9,...,0,0,0,0,0,1,0,0,0,0


In [30]:
for i in range(n_zones):
    plt.plot(df_all.logx[df_all['zone' + str(i)] == 1], df_all.logy[df_all['zone' + str(i)] == 1], color[i])

plt.show()

In [525]:
df_all.latitude.describe()

count    124011.000000
mean         40.737641
std           0.744337
min           0.000000
25%          40.728000
50%          40.751700
75%          40.774300
max          44.883500
Name: latitude, dtype: float64

In [536]:
df_all2 = df_all.copy()
percentile = 1
llimit = np.percentile(df_all.latitude.values, percentile)
ulimit = np.percentile(df_all.latitude.values, 100 - percentile)
print(llimit, ulimit)
df_all2.loc[df_all2['latitude']<llimit, 'latitude'] = llimit
df_all2.loc[df_all2['latitude']>ulimit, 'latitude'] = ulimit
df_all2.latitude.plot(kind='hist', bins = 100)
plt.show()
#1     40.6885 40.81935
#0.01 40.639819 40.8614

40.63981 40.8614


In [537]:
llimit = np.percentile(df_all.longitude.values, percentile)
ulimit = np.percentile(df_all.longitude.values, 100 - percentile)
df_all2.loc[df_all2['longitude']<llimit, 'longitude'] = llimit
df_all2.loc[df_all2['longitude']>ulimit, 'longitude'] = ulimit
df_all2.longitude.plot(kind='hist', bins = 100)
plt.show()

In [538]:
plt.plot(df_all2['latitude'], df_all2['longitude'], 'ro')
plt.show()

In [532]:
kmeans = KMeans(n_clusters=28, random_state=0).fit(df_all2[['latitude', 'longitude']])

In [533]:
color = ['bo', 'ro', 'yo', 'go', 'co', 'mo', 'ko', 
         'bs', 'rs', 'ys', 'gs', 'cs', 'ms', 'ks', 
         'b^', 'r^', 'y^', 'g^', 'c^', 'm^', 'k^',
         'b-', 'r-', 'y-', 'g-', 'c-', 'm-', 'k-']
for i in range(28):
    plt.plot(df_all2.latitude[kmeans.labels_ == i], df_all2.longitude[kmeans.labels_ == i], color[i])
plt.show()

In [469]:
X_train = df_all[:df_train.shape[0]].values
X_test = df_all[df_train.shape[0]:].values

In [ ]:
df_all['bathrooms'].describe()
df_all['bathrooms'].plot(kind='hist')
plt.show()

In [108]:
df_all['bedrooms'].describe()
df_all['bedrooms'].plot(kind='hist')
plt.show()

In [619]:
df_all['price'].describe()
df_all[df_all['price'] < 100000]['price'].plot(kind='hist', bins=100)
plt.show()

In [621]:
df_all[df_all['price'] > 100000]

,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,price,street_address,photos_count,...,zone10,zone11,zone12,zone13,zone14,zone15,zone16,zone17,zone18,zone19
listing_id,,,,,,,,,,,,,,,,,,,,,
7208794,1.0,2,4238,15003,40.8011,-73.9480,3611,1150000,16298,0,...,0,0,0,0,0,0,0,0,0,0
6857401,0.0,0,8456,9241,40.7161,-74.0080,3749,135000,3276,20,...,0,0,0,0,0,0,0,0,0,0
7208764,1.0,2,9327,11768,40.7299,-74.0071,3611,4490000,17314,0,...,0,0,0,1,0,0,0,0,0,0
7122037,1.0,0,8764,13536,40.7323,-73.8761,1072,111111,21044,1,...,0,0,0,0,0,0,0,0,0,0
7013217,1.0,1,2473,15369,40.7676,-73.9844,2452,1070000,14261,0,...,0,0,0,0,1,0,0,0,0,0
7036279,1.0,1,2473,15369,40.7676,-73.9844,424,1070000,14261,7,...,0,0,0,0,1,0,0,0,0,0
6943825,1.0,1,10683,8593,40.7067,-74.0112,1188,383600,10246,12,...,0,0,0,0,0,0,0,0,0,0
7197770,1.0,1,2473,15369,40.7676,-73.9844,3732,1070000,14261,7,...,0,0,0,0,1,0,0,0,0,0
7147232,1.0,1,2473,15369,40.7676,-73.9844,2192,1070000,14261,0,...,0,0,0,0,1,0,0,0,0,0


In [625]:
np.log(df_all['price']).plot(kind='hist', bins=100)
plt.show()

In [115]:
df_all['photos_count'].plot(kind='hist')
plt.show()

In [116]:
df_all['features_count'].plot(kind='hist')
plt.show()

In [117]:
df_all['description_words_count'].plot(kind='hist')
plt.show()

In [118]:
df_all['created_month'].plot(kind='hist')
plt.show()

In [119]:
df_all['created_month'].describe()

count    124011.000000
mean          5.015386
std           0.825266
min           4.000000
25%           4.000000
50%           5.000000
75%           6.000000
max           6.000000
Name: created_month, dtype: float64

In [56]:
df_all['created_day_of_year'].plot(kind='hist', bins = 30)
plt.show()

In [71]:
ohe = OneHotEncoder(sparse=False)
df_all_ohe = ohe.fit_transform(df_all.created_weekday.reshape(-1, 1)) 

In [79]:
df_all_ohe

124011

In [80]:
for i in range(df_all_ohe.shape[1]):
    df_all['ohe' + str(i)] = df_all_ohe[:, i]

In [81]:
df_all

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,longitude,manager_id,...,logx,logy,radius,ohe0,ohe1,ohe2,ohe3,ohe4,ohe5,ohe6
0,1.5,3,3797,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,12282,[],40.7145,-73.9425,1568,...,3.706584,3.260306,-7.446341,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,2,8986,2016-06-12 12:19:27,,9080,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,-73.9667,1988,...,3.708552,3.259376,-5.639716,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,1,8889,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",13719,"[Laundry In Building, Dishwasher, Hardwood Flo...",40.7388,-74.0018,3733,...,3.707181,3.258027,-5.898510,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1,1848,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,10866,"[Hardwood Floors, No Fee]",40.7539,-73.9677,282,...,3.707552,3.259338,-7.421907,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,15072,[Pre-War],40.8241,-73.9493,2618,...,3.709273,3.260045,-4.896160,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,2.0,4,2544,2016-04-19 04:24:47,,15194,[],40.7429,-74.0028,3081,...,3.707282,3.257989,-5.851502,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,1.0,2,2711,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,14964,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",40.8012,-73.9660,12,...,3.708711,3.259403,-5.445840,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,2.0,1,169,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",15223,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",40.7427,-73.9957,3959,...,3.707277,3.258262,-6.134814,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,1,7635,2016-04-20 02:36:35,<p><a website_redacted,11646,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",40.8234,-73.9457,3339,...,3.709255,3.260183,-4.910527,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,2.0,4,0,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,6459,"[Dishwasher, Hardwood Floors]",40.7278,-73.9808,633,...,3.706911,3.258835,-6.830186,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
df_all['building_id'].describe()

count    124011.000000
mean       4974.676480
std        3738.956122
min           0.000000
25%        1253.000000
50%        4999.000000
75%        8195.000000
max       11634.000000
Name: building_id, dtype: float64

In [289]:
value_counts = df_all['building_id'].value_counts()
value_counts

0        20664
6806       705
810        546
8528       522
5859       510
9393       459
11193      457
9139       410
10658      397
834        385
3990       373
6439       367
7088       362
9234       356
9466       349
10090      319
7942       318
3886       314
1126       313
1283       308
438        308
5532       306
10712      303
7253       295
4963       293
2154       291
5854       286
8910       283
6701       283
7444       276
         ...  
3733         1
4462         1
10601        1
5846         1
8552         1
10000        1
6447         1
8488         1
10857        1
4718         1
9424         1
6767         1
11113        1
9064         1
5334         1
7383         1
6959         1
11537        1
3349         1
812          1
7447         1
9552         1
11601        1
11049        1
6895         1
6831         1
9616         1
3477         1
9680         1
3998         1
Name: building_id, dtype: int64

In [255]:
df_all

,bathrooms,bedrooms,building_id,display_address,latitude,listing_id,longitude,manager_id,price,street_address,...,created_day,created_day_of_year,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words
0,1.5,3,3797,12282,40.7145,7211212,-73.9425,1568,3000,23484,...,24,176,7,4,3.706584,3.260306,-7.446341,5,0,95
1,1.0,2,8986,9080,40.7947,7150865,-73.9667,1988,5465,23680,...,12,164,12,6,3.708552,3.259376,-5.639716,11,5,9
2,1.0,1,8889,13719,40.7388,6887163,-74.0018,3733,2850,9827,...,17,108,3,6,3.707181,3.258027,-5.898510,8,4,94
3,1.0,1,1848,10866,40.7539,6888711,-73.9677,282,3275,14237,...,18,109,2,0,3.707552,3.259338,-7.421907,3,2,80
4,1.0,4,0,15072,40.8241,6934781,-73.9493,2618,3350,19227,...,28,119,1,3,3.709273,3.260045,-4.896160,3,1,68
5,2.0,4,2544,15194,40.7429,6894514,-74.0028,3081,7995,15263,...,19,110,4,1,3.707282,3.257989,-5.851502,5,0,9
6,1.0,2,2711,14964,40.8012,6930771,-73.9660,12,3600,7466,...,27,118,3,2,3.708711,3.259403,-5.445840,10,8,87
7,2.0,1,169,15223,40.7427,6867392,-73.9957,3959,5645,4181,...,13,104,6,2,3.707277,3.258262,-6.134814,5,8,134
8,1.0,1,7635,11646,40.8234,6898799,-73.9457,3339,1725,21836,...,20,111,2,2,3.709255,3.260183,-4.910527,5,4,4
9,2.0,4,0,6459,40.7278,6814332,-73.9808,633,5800,20000,...,2,93,2,5,3.706911,3.258835,-6.830186,9,2,166


In [242]:
df_counts

,count_building
building_id,
0,20664
1,1
2,7
3,2
4,1
5,20
6,2
7,1
8,3


In [302]:
#df_all.join(df_counts, lsuffix='building_id', rsuffix='building_id')
df_all2 = pd.merge(df_all, pd.DataFrame(value_counts), left_on='building_id', right_index=True).sort_index()
#df_all2 = pd.merge(df_all, df_counts, left_on='building_id', right_index=True)

In [290]:
df_counts = pd.DataFrame(df_all.groupby('building_id').size(), columns = ['building_count'])

In [286]:
df_all.loc[df_all.building_id == 0, 'building_id_y'] = 0

In [293]:
df_all = df_all.drop(['building_id_x'], axis=1)

In [303]:
df_all2

,building_id,bathrooms,bedrooms,building_id_x,display_address,latitude,listing_id,longitude,manager_id,price,...,created_day_of_year,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words,building_id_y
0,3797,1.5,3,3797,12282,40.7145,7211212,-73.9425,1568,3000,...,176,7,4,3.706584,3.260306,-7.446341,5,0,95,5
1,8986,1.0,2,8986,9080,40.7947,7150865,-73.9667,1988,5465,...,164,12,6,3.708552,3.259376,-5.639716,11,5,9,51
2,8889,1.0,1,8889,13719,40.7388,6887163,-74.0018,3733,2850,...,108,3,6,3.707181,3.258027,-5.898510,8,4,94,131
3,1848,1.0,1,1848,10866,40.7539,6888711,-73.9677,282,3275,...,109,2,0,3.707552,3.259338,-7.421907,3,2,80,236
4,0,1.0,4,0,15072,40.8241,6934781,-73.9493,2618,3350,...,119,1,3,3.709273,3.260045,-4.896160,3,1,68,20664
5,2544,2.0,4,2544,15194,40.7429,6894514,-74.0028,3081,7995,...,110,4,1,3.707282,3.257989,-5.851502,5,0,9,34
6,2711,1.0,2,2711,14964,40.8012,6930771,-73.9660,12,3600,...,118,3,2,3.708711,3.259403,-5.445840,10,8,87,40
7,169,2.0,1,169,15223,40.7427,6867392,-73.9957,3959,5645,...,104,6,2,3.707277,3.258262,-6.134814,5,8,134,74
8,7635,1.0,1,7635,11646,40.8234,6898799,-73.9457,3339,1725,...,111,2,2,3.709255,3.260183,-4.910527,5,4,4,37
9,0,2.0,4,0,6459,40.7278,6814332,-73.9808,633,5800,...,93,2,5,3.706911,3.258835,-6.830186,9,2,166,20664


In [214]:
le = LabelEncoder()
le.fit(df_all.building_id)
le.classes_

array([    0,     1,     2, ..., 11632, 11633, 11634], dtype=int64)

In [216]:
le = LabelEncoder()
le.fit(counts3)
le.classes_

array([    0,     1,     2, ..., 11632, 11633, 11634], dtype=int64)

In [304]:
df_all['manager_id'].describe()

count    124011.000000
mean       2251.196176
std        1267.110571
min           0.000000
25%        1139.000000
50%        2236.000000
75%        3438.000000
max        4398.000000
Name: manager_id, dtype: float64

In [305]:
df_all['manager_id'].plot(kind='hist')
plt.show()

In [306]:
df_all[df_all.manager_id == 0]


,bathrooms,bedrooms,building_id,display_address,latitude,listing_id,longitude,manager_id,price,street_address,...,created_day,created_day_of_year,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words
1496,2.0,2,6286,9926,40.7647,6919657,-73.9590,0,4295,14817,...,24,115,3,6,3.707817,3.259672,-7.101689,6,4,150
2612,1.0,1,6701,8046,40.7637,6948112,-73.9846,0,2885,24167,...,30,121,5,5,3.707792,3.258689,-6.257658,4,5,129
2842,1.0,0,6467,9608,40.7448,6888968,-73.9773,0,2495,8277,...,18,109,2,0,3.707328,3.258969,-7.096783,3,4,123
12366,1.0,0,8212,14582,40.7911,7178288,-73.9735,0,2167,7628,...,17,169,6,4,3.708464,3.259115,-5.673137,4,9,133
12431,1.0,2,8204,2752,40.7806,7178219,-73.9470,0,2800,5545,...,17,169,6,4,3.708206,3.260133,-6.291798,4,3,130
12991,1.0,0,1115,14058,40.7441,7145678,-73.9964,0,2400,7190,...,12,164,3,6,3.707311,3.258235,-6.097929,3,8,114
14051,1.0,0,1115,14058,40.7441,7130221,-73.9964,0,2225,7190,...,9,161,3,3,3.707311,3.258235,-6.097929,5,7,105
18681,1.0,1,9466,14122,40.7530,7123565,-73.9958,0,3175,15583,...,8,160,2,2,3.707529,3.258258,-6.038261,6,9,157
27577,1.0,1,2508,10093,40.7741,7063642,-73.9554,0,1995,9442,...,25,146,3,2,3.708047,3.259810,-6.596693,4,0,117
30180,1.0,0,5650,14074,40.7479,7073940,-74.0005,0,2595,16502,...,27,148,2,4,3.707404,3.258077,-5.909708,4,5,129


In [510]:
value_counts = df_all['manager_id'].value_counts()
df_all = pd.merge(df_all, pd.DataFrame(value_counts), left_on='manager_id', right_index=True).sort_index()
df_all = df_all.drop(['manager_id_x'], axis=1)    
df_all.loc[df_all.manager_id == 0, 'manager_id_y'] = 0

In [511]:
df_all

,manager_id,bathrooms,bedrooms,building_id,description,display_address,features,latitude,listing_id,longitude,...,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words,manager_skill,manager_id_y
0,5ba989232d0489da1b5f2c45f6688adc,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,,40.7145,7211212,-73.9425,...,7,4,3.706584,3.260306,-7.446341,5,0,95,0.744444,235
1,7533621a882f71e25173b27e3139d83d,1.0,2,c5c8a357cba207596b04d1afd1e4f130,,Columbus Avenue,Doorman Elevator Fitness_Center Cats_Allowed D...,40.7947,7150865,-73.9667,...,12,6,3.708552,3.259376,-5.639716,11,5,9,0.988372,194
2,d9039c43983f6e564b1482b273bd7b01,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,"Top Top West Village location, beautiful Pre-w...",W 13 Street,Laundry_In_Building Dishwasher Hardwood_Floors...,40.7388,6887163,-74.0018,...,3,6,3.707181,3.258027,-5.898510,8,4,94,0.694030,314
3,1067e078446a7897d2da493d2f741316,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,Building Amenities - Garage - Garden - fitness...,East 49th Street,Hardwood_Floors No_Fee,40.7539,6888711,-73.9677,...,2,0,3.707552,3.259338,-7.421907,3,2,80,0.942408,470
4,98e13ad4b495b9613cef886d79a6291f,1.0,4,0,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,Pre-War,40.8241,6934781,-73.9493,...,1,3,3.709273,3.260045,-4.896160,3,1,68,1.000000,51
5,b209e2c4384a64cc307c26759ee0c651,2.0,4,38a913e46c94a7f46ddf19b756a9640c,,West 18th Street,,40.7429,6894514,-74.0028,...,4,1,3.707282,3.257989,-5.851502,5,0,9,0.909524,538
6,01287194f20de51872e81f660def4784,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,Stunning unit with a great location and lots o...,West 107th Street,prewar elevator Dogs_Allowed Cats_Allowed LOWR...,40.8012,6930771,-73.9660,...,3,2,3.708711,3.259403,-5.445840,10,8,87,1.000000,20
7,e6472c7237327dd3903b3d6f6a94515a,2.0,1,0372927bcb6a0949613ef5bf893bbac7,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,Doorman Elevator Pre-War Terrace Laundry_in_Un...,40.7427,6867392,-73.9957,...,6,2,3.707277,3.258262,-6.134814,5,8,134,0.822345,6387
8,c1a6598437b7db560cde66e5a297a53f,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,<p><a website_redacted,Hamilton Terrace,Cats_Allowed Dogs_Allowed Elevator Laundry_In_...,40.8234,6898799,-73.9457,...,2,2,3.709255,3.260183,-4.910527,5,4,4,0.883721,207
9,23a01ea7717b38875f5b070282d1b9d2,2.0,4,0,This is a spacious four bedroom with every bed...,522 E 11th,Dishwasher Hardwood_Floors,40.7278,6814332,-73.9808,...,2,5,3.706911,3.258835,-6.830186,9,2,166,0.538462,21


In [482]:
df_all[:df_train.shape[0]].groupby('manager_id').mean().shape

(3481, 17)

In [484]:
df_all[df_train.shape[0]:].groupby('manager_id').mean().shape

(3851, 17)

In [483]:
df_all.groupby('manager_id').count().shape

(4399, 23)

In [563]:
temp = pd.concat([df_all[:df_train.shape[0]].manager_id, pd.get_dummies(y_train)], axis = 1).groupby('manager_id').mean()
temp.columns = ['high_frac', 'low_frac', 'medium_frac']
#this is equivalent of number of reviews
temp['manager_listings'] = df_all[:df_train.shape[0]].groupby('manager_id').count().iloc[:,1]
#this is equivalent to star rating (0, 1 or 2 stars)
temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']
#lower the rating for fewer listings
temp['manager_skill'] = temp.manager_skill*expit((temp.manager_listings)/4)
temp = temp['manager_skill']
temp.describe()

count    3481.000000
mean        0.677096
std         0.260735
min         0.000000
25%         0.562177
50%         0.679179
75%         0.851953
max         1.635149
Name: manager_skill, dtype: float64

In [514]:
temp = pd.concat([df_all[:df_train.shape[0]].manager_id, pd.get_dummies(y_train)], axis = 1).groupby('manager_id').mean()
temp.columns = ['high_frac','low_frac', 'medium_frac']
temp['count'] = df_all[:df_train.shape[0]].groupby('manager_id').count().iloc[:,1]
temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']
print(temp.shape)
temp = temp[['manager_skill', 'count']]
#temp.describe()
temp

(3481, 5)


,manager_skill,count
manager_id,,
0000abd7518b94c35a90d64b56fbf3e6,0.333333,12
001ce808ce1720e24a9510e014c69707,1.000000,10
003fc4e9a70053082f131b1054966aaf,1.000000,1
00607a02f6efd9c6c7c588826e471ee9,0.000000,2
00995ff28d79127ed2dca8320e9e7d09,0.666667,3
00a8d77892cab18fffaa22a751f1f8eb,1.000000,3
00f0b3346ba8379b26b95be8a19d46b1,1.000000,1
00f526d80353a50a52bbc26919e7ed5a,1.000000,1
01074bb974b125b65fa1b11a64c65135,1.000000,3


In [546]:
from scipy.special import expit


In [547]:
x = [x for x in range(1, 51)]
y = [expit((y - 1)/4) for y in x]
y

In [554]:
df1 = pd.DataFrame({'x': x, 'y' : y})
df1

,x,y
0,1,0.500000
1,2,0.562177
2,3,0.622459
3,4,0.679179
4,5,0.731059
5,6,0.777300
6,7,0.817574
7,8,0.851953
8,9,0.880797
9,10,0.904651


In [557]:
df1.y*expit((df1.x - 1)/4)

0     0.250000
1     0.316042
2     0.387456
3     0.461284
4     0.534447
5     0.604195
6     0.668428
7     0.725824
8     0.775803
9     0.818393
10    0.854038
11    0.883437
12    0.907397
13    0.926740
14    0.942235
15    0.954573
16    0.964351
17    0.972071
18    0.978147
19    0.982919
20    0.986659
21    0.989587
22    0.991876
23    0.993665
24    0.995061
25    0.996150
26    0.997000
27    0.997662
28    0.998179
29    0.998581
30    0.998895
31    0.999139
32    0.999329
33    0.999478
34    0.999593
35    0.999683
36    0.999753
37    0.999808
38    0.999850
39    0.999883
40    0.999909
41    0.999929
42    0.999945
43    0.999957
44    0.999967
45    0.999974
46    0.999980
47    0.999984
48    0.999988
49    0.999990
dtype: float64

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.0,2.000000,3.000000,4.000000,5.000000,6.0000,7.000000,8.000000,9.000000,10.000000,...,41.000000,42.000000,43.000000,44.000000,45.000000,46.000000,47.00000,48.000000,49.000000,50.000000
1,0.5,0.562177,0.622459,0.679179,0.731059,0.7773,0.817574,0.851953,0.880797,0.904651,...,0.999955,0.999965,0.999972,0.999979,0.999983,0.999987,0.99999,0.999992,0.999994,0.999995


In [537]:
df_all2 = df_all.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')

In [503]:
df_all2 = df_all2.fillna(0)

In [506]:
df_all['manager_skill'] = df_all2['manager_skill']

In [507]:
df_all

,bathrooms,bedrooms,building_id,description,display_address,features,latitude,listing_id,longitude,manager_id,...,created_day_of_year,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words,manager_skill
0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,...,176,7,4,3.706584,3.260306,-7.446341,5,0,95,0.744444
1,1.0,2,c5c8a357cba207596b04d1afd1e4f130,,Columbus Avenue,Doorman Elevator Fitness_Center Cats_Allowed D...,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,...,164,12,6,3.708552,3.259376,-5.639716,11,5,9,0.988372
2,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,"Top Top West Village location, beautiful Pre-w...",W 13 Street,Laundry_In_Building Dishwasher Hardwood_Floors...,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,...,108,3,6,3.707181,3.258027,-5.898510,8,4,94,0.694030
3,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,Building Amenities - Garage - Garden - fitness...,East 49th Street,Hardwood_Floors No_Fee,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,...,109,2,0,3.707552,3.259338,-7.421907,3,2,80,0.942408
4,1.0,4,0,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,Pre-War,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,...,119,1,3,3.709273,3.260045,-4.896160,3,1,68,1.000000
5,2.0,4,38a913e46c94a7f46ddf19b756a9640c,,West 18th Street,,40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,...,110,4,1,3.707282,3.257989,-5.851502,5,0,9,0.909524
6,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,Stunning unit with a great location and lots o...,West 107th Street,prewar elevator Dogs_Allowed Cats_Allowed LOWR...,40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,...,118,3,2,3.708711,3.259403,-5.445840,10,8,87,1.000000
7,2.0,1,0372927bcb6a0949613ef5bf893bbac7,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,Doorman Elevator Pre-War Terrace Laundry_in_Un...,40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,...,104,6,2,3.707277,3.258262,-6.134814,5,8,134,0.822345
8,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,<p><a website_redacted,Hamilton Terrace,Cats_Allowed Dogs_Allowed Elevator Laundry_In_...,40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,...,111,2,2,3.709255,3.260183,-4.910527,5,4,4,0.883721
9,2.0,4,0,This is a spacious four bedroom with every bed...,522 E 11th,Dishwasher Hardwood_Floors,40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,...,93,2,5,3.706911,3.258835,-6.830186,9,2,166,0.538462


In [465]:
y_train

array([1, 2, 0, ..., 2, 2, 2], dtype=int64)

In [434]:
df_train[df_train.manager_id == '00a8d77892cab18fffaa22a751f1f8eb']

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
listing_id,,,,,,,,,,,,,,,
6880599,2.0,4,0,2016-04-16 01:12:38,NO FEENewlyrenovated building with Condo finis...,Halsey Street,"[Pre-War, No Fee]",low,40.6890,6880599,-73.9117,00a8d77892cab18fffaa22a751f1f8eb,[https://photos.renthop.com/2/6880599_5e7705df...,3650,1106 Halsey Street
6961681,1.0,1,0,2016-05-04 01:29:21,**JUST LISTED** Massive Floor through home (87...,East 12th Street,[Pre-War],low,40.7297,6961681,-73.9823,00a8d77892cab18fffaa22a751f1f8eb,[https://photos.renthop.com/2/6961681_e516c523...,3595,423 East 12th Street
6939611,1.0,1,0,2016-04-29 01:25:31,**NO BROKER FEE** Available June 1st! OPEN LOF...,Flushing Avenue,"[Pre-War, No Fee, Dogs Allowed, Cats Allowed]",low,40.7019,6939611,-73.9362,00a8d77892cab18fffaa22a751f1f8eb,[https://photos.renthop.com/2/6939611_714290ee...,2295,888 Flushing Avenue


In [450]:
df_all

,bathrooms,bedrooms,building_id,description,display_address,features,latitude,listing_id,longitude,manager_id,...,created_day,created_day_of_year,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words
0,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan,,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,...,24,176,7,4,3.706584,3.260306,-7.446341,5,0,95
1,1.0,2,c5c8a357cba207596b04d1afd1e4f130,,Columbus,doorman elevator fitness_center cats_allowed d...,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,...,12,164,12,6,3.708552,3.259376,-5.639716,11,5,9
2,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,"Top Top West Village location, beautiful Pre-w...",W 13,laundry_in_building dishwasher hardwood_floors...,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,...,17,108,3,6,3.707181,3.258027,-5.898510,8,4,94
3,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,Building Amenities - Garage - Garden - fitness...,East 49th,hardwood_floors no_fee,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,...,18,109,2,0,3.707552,3.259338,-7.421907,3,2,80
4,1.0,4,0,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd,pre-war,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,...,28,119,1,3,3.709273,3.260045,-4.896160,3,1,68
5,2.0,4,38a913e46c94a7f46ddf19b756a9640c,,West 18th,,40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,...,19,110,4,1,3.707282,3.257989,-5.851502,5,0,9
6,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,Stunning unit with a great location and lots o...,West 107th,prewar elevator dogs_allowed cats_allowed lowr...,40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,...,27,118,3,2,3.708711,3.259403,-5.445840,10,8,87
7,2.0,1,0372927bcb6a0949613ef5bf893bbac7,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st,doorman elevator pre-war terrace laundry_in_un...,40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,...,13,104,6,2,3.707277,3.258262,-6.134814,5,8,134
8,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,<p><a website_redacted,Hamilton Terrace,cats_allowed dogs_allowed elevator laundry_in_...,40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,...,20,111,2,2,3.709255,3.260183,-4.910527,5,4,4
9,2.0,4,0,This is a spacious four bedroom with every bed...,522 E 11th,dishwasher hardwood_floors,40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,...,2,93,2,5,3.706911,3.258835,-6.830186,9,2,166


In [411]:
#df_all['features'] = df_all['features'].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
df_all['features'] = df_all['features'].apply(lambda x: x.lower())
print(df_all['features'].head())



0                                                     
1    doorman elevator fitness_center cats_allowed d...
2    laundry_in_building dishwasher hardwood_floors...
3                               hardwood_floors no_fee
4                                              pre-war
Name: features, dtype: object


In [413]:
df_all.features.describe()
#unique 17378

count     124011
unique     17264
top             
freq        8135
Name: features, dtype: object

In [416]:
df_all[:df_train.shape[0]]['features']


0                                                         
1        doorman elevator fitness_center cats_allowed d...
2        laundry_in_building dishwasher hardwood_floors...
3                                   hardwood_floors no_fee
4                                                  pre-war
5                                                         
6        prewar elevator dogs_allowed cats_allowed lowr...
7        doorman elevator pre-war terrace laundry_in_un...
8        cats_allowed dogs_allowed elevator laundry_in_...
9                               dishwasher hardwood_floors
10                                                        
11       prewar dishwasher highrise roofdeck eat_in_kit...
12             doorman elevator laundry_in_building no_fee
13       swimming_pool doorman fitness_center no_fee do...
14       elevator multi-level laundry_in_building dishw...
15       doorman elevator fitness_center laundry_in_bui...
16                                         hardwood_floo

In [414]:
tfidf = CountVectorizer(stop_words='english', max_features=5000)
tr_sparse = tfidf.fit_transform(df_all[:df_train.shape[0]]['features'])
te_sparse = tfidf.transform(df_all[df_train.shape[0]:]['features'])

In [415]:
tfidf.get_feature_names()
len(tfidf.get_feature_names())
--1649

1649

In [99]:
print(tr_sparse.shape)
print(te_sparse.shape)

(49352, 200)
(74659, 200)


In [91]:
df_all = df_all.drop(['created', 'features', 'description', 'photos'], axis=1)

In [97]:
X_train = df_all[:df_train.shape[0]].values
X_test = df_all[df_train.shape[0]:].values

In [103]:
X_train = sparse.hstack([X_train, tr_sparse])
X_test = sparse.hstack([X_test, te_sparse])

In [407]:
tfidf2 = CountVectorizer(stop_words='english', max_features=20)
tr_sparse2 = tfidf2.fit_transform(df_all[:df_train.shape[0]]['description'])


In [408]:
tfidf2.get_feature_names()
#len(tfidf2.get_feature_names())

['apartment',
 'appliances',
 'bedroom',
 'br',
 'building',
 'com',
 'features',
 'floors',
 'hardwood',
 'kagglemanager',
 'kitchen',
 'large',
 'laundry',
 'new',
 'renovated',
 'renthop',
 'room',
 'space',
 'stainless',
 'website_redacted']

In [402]:
sum(tr_sparse2).todense().shape

(1, 5000)

In [364]:
#df_all[['display_address', 'street_address']]
df_all[df_all.display_address.str.contains('Av')][['display_address', 'street_address']]


,display_address,street_address
130,Ave A,115 Ave A
177,Ave. A,121 Ave. A
933,Ave. B,44 Ave. B
3025,Ave B,212 Ave B
3536,Ave. C,321 Ave. C
4338,Ave. B,44 Ave. B
4340,Ave. C,153 Ave. C
5060,Ave B,220 Ave B
5610,"Ave Of The Americas,","990 Ave of the Americas,"
6444,"Ave Of The Americas,","1020 Ave of the Americas,"


In [360]:
df_all.display_address.describe()
#count       124011
#unique       16068

count       124011
unique       13116
top       Broadway
freq          1195
Name: display_address, dtype: object

In [362]:
#df_all[df_all.display_address == 'Broadway']
#df_all.display_address.str.findall('Broadway') != []
df_all[df_all.display_address.str.contains('Av')]


,bathrooms,bedrooms,building_id,description,display_address,features,latitude,listing_id,longitude,manager_id,...,created_day,created_day_of_year,created_hour,created_weekday,logx,logy,radius,num_photos,num_features,num_description_words
130,1.0,3,0,No Fee** Are you looking for that 3 Bedroom w/...,Ave A,Laundry_in_Unit Dishwasher Hardwood_Floors No_...,40.7264,6837789,-73.9836,17c72c26cc226ed1106689a06cfe5bce,...,7,98,2,3,3.706877,3.258727,-6.650280,5,6,275
177,1.0,3,5174b8948f377ca7ec6d4bf431b7d5f4,This enormous true 3br in the East Village is ...,Ave. A,Hardwood_Floors,40.7265,6932079,-73.9835,92aa3b535f48ec05903a3b7fcdafd411,...,27,118,5,2,3.706879,3.258731,-6.657309,3,1,89
933,1.0,1,cd64e65d632c7f5d152d6c01dad5ac65,Beautifully renovated 1 bdrm apt. located in t...,Ave. B,,40.7230,6933329,-73.9829,1067e078446a7897d2da493d2f741316,...,27,118,6,2,3.706793,3.258754,-6.619337,3,0,77
3025,1.0,3,de08ea7a52b1fdccae697fd90dd761a9,"Large, bright true 3 bedroom apartment with qu...",Ave B,Dishwasher Hardwood_Floors Dogs_Allowed Cats_A...,40.7287,6918384,-73.9788,a2745e4ddab320a1fde31d4bbbdbba47,...,24,115,2,6,3.706933,3.258912,-6.967197,5,4,53
3536,1.0,2,0,NO FEE<br /><br />You will noty be able to fin...,Ave. C,Dining_Room Dishwasher Hardwood_Floors No_Fee ...,40.7320,6889774,-73.9745,91f8a99bd7c5f2a6d50c04ae3972364d,...,18,109,3,0,3.707014,3.259077,-7.323095,6,6,132
4338,1.0,1,cd64e65d632c7f5d152d6c01dad5ac65,Beautifully renovated 1 bdrm apt. located in t...,Ave. B,,40.7230,6875986,-73.9829,1067e078446a7897d2da493d2f741316,...,15,106,2,4,3.706793,3.258754,-6.619337,3,0,77
4340,1.0,1,5a5b40ea980ffd03968fe45e69964e8c,Cozy one bedroom located near great dining and...,Ave. C,Hardwood_Floors,40.7257,6820184,-73.9780,1067e078446a7897d2da493d2f741316,...,4,95,2,0,3.706859,3.258942,-6.950048,5,1,48
5060,1.5,2,c94cd983d0e68fc9885591f216f4e7e6,Panoramic views of city skylines located in th...,Ave B,Dining_Room Balcony Loft Dishwasher Hardwood_F...,40.7290,6839011,-73.9785,0532e99449bc0266d211c5ab9e820f48,...,7,98,4,3,3.706940,3.258923,-6.991695,5,5,132
5610,1.0,2,80911ce8a425daf4989ea8a4bccc41a7,Amazing renovated convertible two bedroom in a...,"Ave Of The Americas,",private-outdoor-space building-common-outdoor-...,40.7513,6909836,-73.9862,d485144d5b5f08efdcbd473396b972bc,...,22,113,2,4,3.707488,3.258627,-6.477073,11,16,70
6444,1.0,2,cae2cf7540a12563195993d1043a6f85,Luxury residence located at the very heart of ...,"Ave Of The Americas,",private-outdoor-space building-common-outdoor-...,40.7524,6875076,-73.9857,9d9e32a8be582241565541ec8073a1c8,...,15,106,2,4,3.707515,3.258646,-6.480495,5,16,86


In [358]:
df_all['display_address'] = df_all.display_address.str.replace('Avenue', '')
df_all['display_address'] = df_all.display_address.str.replace(' Ave', '')
df_all['display_address'] = df_all.display_address.str.replace('Street', '')
df_all['display_address'] = df_all.display_address.str.replace('St.', '')
df_all['display_address'] = df_all.display_address.str.replace(' St', '')
df_all['display_address'] = df_all.display_address.str.rstrip()

In [379]:
df_all.street_address.describe()

count            124011
unique            25766
top       3333 Broadway
freq                449
Name: street_address, dtype: object

In [380]:
#df_all.street_address.apply(lambda x: x.split(" ")[0])
df_all.street_address.apply(lambda x: x.split(" ")[0]).describe()

count     124011
unique      3466
top          200
freq        2317
Name: street_address, dtype: object

In [394]:
df_all.description[11]

"Check out this one bedroom apartment in a great location of Washington Heights.  This apartment is just a block from everything you need!  Also within walking distance to Columbia Presbyterian! This apartment won't last long on the market! Call now! Call/ Email Heidi to tour apartments in the area and anything you see here on Bohemia kagglemanager@renthop.com 189-560-4844 APARTMENT:* Dishwasher* Queen Size Sunny Bedroom* Spacious Living room* Hardwood Floors* Stainless Steel appliances * Subway (A/C &amp; 1 Trains)* Kind &amp; friendly Broker :)Located by W 173 &amp; Fort Washington  Call today and I can show you everything Bohemia has to offer189-560-4844 kagglemanager@renthop.com"

In [132]:
y_train

array([2, 1, 0, ..., 1, 1, 1], dtype=int64)

In [126]:
plt.hist(y_train)
plt.show()

In [127]:
le.transform(['high', 'medium', 'low'])

array([0, 2, 1], dtype=int64)

In [181]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(49352, 309)
(49352,)
(74659, 309)


In [182]:
np.random.seed(0)

learning_rate, max_depth, ss, cs, gamma, min_child_weight, reg_lambda, reg_alpha = 0.1, 6, 0.7, 0.7, 0, 1, 1, 0
early_stopping_rounds = 100
clf = XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=5000, objective='multi:softprob', subsample=ss, colsample_bytree=cs, gamma=gamma, min_child_weight=min_child_weight, reg_lambda=reg_lambda, reg_alpha=reg_alpha, seed=0)

kf = KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(X_train.shape[0])):
        X_train_train, X_train_test = X_train[dev_index,:], X_train[val_index,:]
        y_train_train, y_train_test = y_train[dev_index], y_train[val_index]
        clf.fit(X_train_train, y_train_train, eval_set=[(X_train_test, y_train_test)], eval_metric='mlogloss', early_stopping_rounds=early_stopping_rounds, verbose=100)
        break


Will train until validation_0 error hasn't decreased in 100 rounds.
[0]	validation_0-mlogloss:1.042445
[100]	validation_0-mlogloss:0.567222
[200]	validation_0-mlogloss:0.552777
[300]	validation_0-mlogloss:0.547953
[400]	validation_0-mlogloss:0.547703
Stopping. Best iteration:
[366]	validation_0-mlogloss:0.547125



In [46]:
np.random.seed(0)
learning_rate, max_depth, ss, cs, gamma, min_child_weight, reg_lambda, reg_alpha = 0.1, 6, 0.7, 0.7, 0, 1, 1, 0
early_stopping_rounds = 100
clf = XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=5000, objective='multi:softprob', subsample=ss, colsample_bytree=cs, gamma=gamma, min_child_weight=min_child_weight, reg_lambda=reg_lambda, reg_alpha=reg_alpha, seed=0)

xgb_options = clf.get_xgb_params()
xgb_options['num_class'] = 3
xgb_options.update({"eval_metric":'mlogloss'})
train_dmatrix = DMatrix(csr_matrix(X_train), label=y_train)

folds = StratifiedKFold(y_train, n_folds=5, shuffle=True)
cv_results = cv(xgb_options, train_dmatrix, clf.n_estimators, early_stopping_rounds=early_stopping_rounds, verbose_eval=100, show_stdv=False, folds=folds)

#1 
#[342] cv-mean:0.5680868	cv-std:0.0030260052478474127
#2 no changes?
#[340] cv-mean:0.5676534	cv-std:0.004004490159808118
#3 +Log(distance)
#[374] cv-mean:0.5667530000000001	cv-std:0.003907353836037892
#4 +Replace incorrect x, y with mean +- 5
#[372] cv-mean:0.5669502000000001	cv-std:0.0034901165825800095
#5 +logx, logy
#[376] cv-mean:0.5670769999999999	cv-std:0.004030059552909871
#6 +square
#[323] cv-mean:0.5672952	cv-std:0.0029957986180649803
#7 +zones
#[327] cv-mean:0.565626	cv-std:0.003298155120669733
#8 +features
#[326] cv-mean:0.5517966	cv-std:0.004730903279501714
#9 ss/cs 0.7
#[305] cv-mean:0.5491194	cv-std:0.005119665168739082
#10 +listing_id
#[325] cv-mean:0.5464508	cv-std:0.0036572058952156396


Will train until cv error hasn't decreased in 100 rounds.
[0]	cv-test-mlogloss:1.037572	cv-train-mlogloss:1.036626
[100]	cv-test-mlogloss:0.5659744	cv-train-mlogloss:0.49203460000000004
[200]	cv-test-mlogloss:0.5507876	cv-train-mlogloss:0.42027279999999995
[300]	cv-test-mlogloss:0.5466428	cv-train-mlogloss:0.3687818
[400]	cv-test-mlogloss:0.5468223999999999	cv-train-mlogloss:0.3263088
Stopping. Best iteration:
[325] cv-mean:0.5464508	cv-std:0.0036572058952156396


In [25]:
np.random.seed(0)
learning_rate, max_depth, ss, cs, gamma, min_child_weight, reg_lambda, reg_alpha = 0.1, 6, 0.5, 0.5, 0, 1, 1, 0
clf = XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=400, objective='multi:softprob', subsample=ss, colsample_bytree=cs, gamma=gamma, min_child_weight=min_child_weight, reg_lambda=reg_lambda, reg_alpha=reg_alpha)
clf.fit(csr_matrix(X_train), y_train)
y_predict = clf.predict_proba(csr_matrix(X_test))
y_predict 

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=400, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.5)

In [28]:
df_out = pd.DataFrame(y_predict)
df_out.columns = ["high", "medium", "low"]
df_out["listing_id"] = df_test.index.values
df_out.to_csv("Output/results.csv", index=False)

In [40]:
df_train.listing_id.describe()

count    4.935200e+04
mean     7.024055e+06
std      1.262746e+05
min      6.811957e+06
25%      6.915888e+06
50%      7.021070e+06
75%      7.128733e+06
max      7.753784e+06
Name: listing_id, dtype: float64

In [41]:
df_train.shape

(49352, 15)

In [42]:
df_test.listing_id.describe()

count    7.465900e+04
mean     7.024001e+06
std      1.264496e+05
min      6.811958e+06
25%      6.915516e+06
50%      7.021738e+06
75%      7.129166e+06
max      7.761779e+06
Name: listing_id, dtype: float64

In [43]:
df_test.shape

(74659, 14)

In [47]:
df_train.listing_id.plot(kind='hist')
df_test.listing_id.plot(kind='hist')
plt.show()

In [55]:
plt.plot(df_train.listing_id, np.zeros(df_train.shape[0]), 'ro')
plt.plot(df_test.listing_id,  np.ones(df_test.shape[0]), 'bo')
plt.plot([7000000], [-1],  'bo')
plt.plot([7000000], [2],  'bo')
plt.show()